In [39]:
from Bio.PDB import PDBParser, PPBuilder, Selection
from Bio import SeqIO
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import pickle
import json
import ast
import torch
import numpy as np

# Merge and Prepare Datasets

In [69]:
dips_csvs = []
for i in range(1, 11):
    path_start = "/home/gridsan/kalyanpa/DNAInteract_shared/kalyan/pepgen_data/partner_seqs_parallel/dips_20_mers_parallel"
    dips_csvs.append(pd.read_csv(f"{path_start}_{i}.csv"))

dips_data = pd.concat(dips_csvs)

column_rename = {
        'pdb_code': 'pdb_name',
        'chain_1': 'partner_chain',
        'chain_2': 'peptide_chain',
        'peptide_sequence': 'peptide_seq',
        'partner_sequence': 'partner_seq',
}
dips_data = dips_data[list(column_rename.keys())].rename(columns=column_rename).dropna()
dips_data['peptide_chain'] = dips_data['peptide_chain'].str.strip()
dips_data['partner_chain'] = dips_data['partner_chain'].str.strip()
dips_data['pdb_id'] = dips_data[['pdb_name', 'peptide_chain', 'partner_chain']] \
                                    .agg('_'.join, axis=1)
dips_data['partner_chain'] = dips_data[['pdb_name', 'partner_chain']] \
                                    .agg('_'.join, axis=1)
dips_data['peptide_chain'] = dips_data[['pdb_name', 'peptide_chain']] \
                                    .agg('_'.join, axis=1)

dips_data = dips_data.dropna()


In [70]:
dips_interfaces = pd.read_csv("interface_pairs.csv").rename(columns={"pdb_id_str": "pdb_id"})

In [71]:
dips_data = dips_data.merge(dips_interfaces[["pdb_id", "interface_pair"]], on="pdb_id", how="left")

In [75]:
dips_data
dips_binding_regions = []
for i, row in dips_data.iterrows():
    if pd.isnull(row.interface_pair):
        dips_binding_regions.append(None)
    else:
        start = row.partner_seq.find(row.interface_pair)
        dips_binding_regions.append(list(range(start, start + len(row.interface_pair))))

In [76]:
dips_data = dips_data.assign(binding_region_indices=dips_binding_regions).drop(columns=["interface_pair"])

In [77]:
propedia_data = pd.read_csv("propedia_data/binding_regions_interfaces.csv").rename(columns={"propedia_id": "pdb_id"})
propedia_data = propedia_data[dips_data.columns]
propedia_data['partner_chain'] = propedia_data[['pdb_name', 'partner_chain']] \
                                    .agg('_'.join, axis=1)
propedia_data['peptide_chain'] = propedia_data[['pdb_name', 'peptide_chain']] \
                                    .agg('_'.join, axis=1)
propedia_data

,pdb_name,partner_chain,peptide_chain,peptide_seq,partner_seq,pdb_id,binding_region_indices
0,148l,148l_E,148l_S,AXXXX,MNIFEMLRIDEGLRLKIYKDTEGYYEIGIGHLLTKSPSLNAAKSEL...,148l_S_E,"[10, 17, 19, 20, 21, 23, 25, 28, 29, 30, 31, 3..."
1,1a07,1a07_A,1a07_C,XEX,SIQAEEWYFGKITRRESERLLLNAENPRGTFLVRESETTKGAYCLS...,1a07_C_A,"[13, 33, 35, 36, 38, 42, 43, 44, 57, 58, 59, 6..."
2,1a07,1a07_B,1a07_D,XEX,IQAEEWYFGKITRRESERLLLNAENPRGTFLVRESETTKGAYCLSV...,1a07_D_B,"[12, 32, 34, 35, 36, 37, 41, 42, 43, 56, 57, 5..."
3,1a08,1a08_A,1a08_C,XEX,SIQAEEWYFGKITRRESERLLLNAENPRGTFLVRESETTKGAYCLS...,1a08_C_A,"[13, 16, 33, 34, 35, 36, 37, 38, 41, 42, 43, 4..."
4,1a08,1a08_B,1a08_D,XEX,IQAEEWYFGKITRRESERLLLNAENPRGTFLVRESETTKGAYCLSV...,1a08_D_B,"[10, 11, 12, 32, 33, 34, 35, 36, 37, 40, 41, 4..."
...,...,...,...,...,...,...,...
19808,8kme,8kme_2,8kme_3,DFEEIPEEXL,IVEGSDAEIGMSPWQVMLFRKSPQELLCGASLISDRWVLTAAHCLL...,8kme_3_2,"[16, 18, 20, 21, 23, 24, 25, 59, 61, 67, 68, 6..."
19809,8kme,8kme_2,8kme_4,XXXLPX,IVEGSDAEIGMSPWQVMLFRKSPQELLCGASLISDRWVLTAAHCLL...,8kme_4_2,"[42, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 9..."
19810,8lpr,8lpr_A,8lpr_P,AAPX,ANIVGGIEYSINNASLCSVGFSVTRGATKGFVTAGHCGTVNATARI...,8lpr_P_A,"[15, 16, 35, 36, 58, 62, 120, 121, 122, 123, 1..."
19811,8pch,8pch_A,8pch_P,EPQNCSAT,YPPSMDWRKKGNFVSPVKNQGSCGSCWTFSTTGALESAVAIATGKM...,8pch_P_A,"[23, 25, 26, 61, 63, 66, 67, 68, 69, 70, 71, 7..."


In [78]:
combined_dataset = pd.concat([propedia_data, dips_data], ignore_index=True)

In [79]:
with open("all_sequences.fasta", "w") as f:
    for i, row in combined_dataset.iterrows():
        f.write(f">{row['pdb_id']}\n")
        f.write(f"{row['partner_seq']}\n")

# Cluster Homologous Sequences

We need to do this for train/test/val split

In [80]:
!mkdir mmseq_output_50

mkdir: cannot create directory 'mmseq_output_50': File exists


In [81]:
!mmseqs/bin/mmseqs easy-linclust all_sequences.fasta mmseq_output_50/clusterRes tmp --min-seq-id 0.50

easy-linclust all_sequences.fasta mmseq_output_50/clusterRes tmp --min-seq-id 0.50 

MMseqs Version:                     	92deb92fb46583b4c68932111303d12dfa121364
Cluster mode                        	0
Max connected component depth       	1000
Similarity type                     	2
Threads                             	96
Compressed                          	0
Verbosity                           	3
Substitution matrix                 	aa:blosum62.out,nucl:nucleotide.out
Add backtrace                       	false
Alignment mode                      	0
Alignment mode                      	0
Allow wrapped scoring               	false
E-value threshold                   	0.001
Seq. id. threshold                  	0.5
Min alignment length                	0
Seq. id. mode                       	0
Alternative alignments              	0
Coverage threshold                  	0.8
Coverage mode                       	0
Max sequence length                 	65535
Compositional bias                  	1

[=================================================================] 100.00% 8.47K 0s 60ms


Time for merging to pref_rescore2: 0h 0m 0s 163ms                 ] 69.67% 5.90K eta 0s       
Time for processing: 0h 0m 1s 195ms
align tmp/15186269317561335732/clu_tmp/979734448373313304/input_step_redundancy tmp/15186269317561335732/clu_tmp/979734448373313304/input_step_redundancy tmp/15186269317561335732/clu_tmp/979734448373313304/pref_rescore2 tmp/15186269317561335732/clu_tmp/979734448373313304/aln --sub-mat aa:blosum62.out,nucl:nucleotide.out -a 0 --alignment-mode 2 --alignment-output-mode 0 --wrapped-scoring 0 -e 0.001 --min-seq-id 0.5 --min-aln-len 0 --seq-id-mode 0 --alt-ali 0 -c 0.8 --cov-mode 0 --max-seq-len 65535 --comp-bias-corr 1 --comp-bias-corr-scale 1 --max-rejected 2147483647 --max-accept 2147483647 --add-self-matches 0 --db-load-mode 0 --pca substitution:1.100,context:1.400 --pcb substitution:4.100,context:5.800 --score-bias 0 --realign 0 --realign-score-bias -0.2 --realign-max-seqs 2147483647 --corr-score-weight 0 --gap-open aa:11,nucl:5 --gap-extend aa:1,nucl:2 --z

In [82]:
clusters = pd.read_csv("mmseq_output_50/clusterRes_cluster.tsv", sep="\t", header=None)

In [83]:
len(clusters[0].unique())

7434

In [84]:
random_seed = 42

train_clusters, test_clusters = train_test_split(clusters[0].unique(), train_size=0.7, test_size=0.3, random_state=random_seed)
test_clusters, val_clusters = train_test_split(test_clusters, train_size=0.5, test_size=0.5, random_state=random_seed)


In [85]:
train_ids = [x.split('|')[0].replace("-", "_") for x in clusters[clusters[0].isin(train_clusters)][1]]
val_ids = [x.split('|')[0].replace("-", "_") for x in clusters[clusters[0].isin(val_clusters)][1]]
test_ids = [x.split('|')[0].replace("-", "_") for x in clusters[clusters[0].isin(test_clusters)][1]]

In [86]:
splits = []
for i, row in combined_dataset.iterrows():
    if row['pdb_id'] in train_ids:
        splits.append("train")
    elif row['pdb_id'] in val_ids:
        splits.append("val")
    elif row['pdb_id'] in test_ids:
        splits.append("test")
    else:
        splits.append(None)

In [87]:
combined_dataset = combined_dataset.assign(split=splits)

In [88]:
train_proportion = sum(combined_dataset['split'] == 'train') / len(combined_dataset)
val_proportion = sum(combined_dataset['split'] == 'val') / len(combined_dataset)
test_proportion = sum(combined_dataset['split'] == 'test') / len(combined_dataset)

print(f"Train proportion: {train_proportion}")
print(f"Val proportion: {val_proportion}")
print(f"Test proportion: {test_proportion}")

Train proportion: 0.7013277081523762
Val proportion: 0.13701038176779584
Test proportion: 0.16166191007982794


In [89]:
combined_dataset.to_csv("pepgen_dataset.csv")

# Manage Embeddings

In [26]:
with open("manu_data_embeddings.pkl", "rb") as f:
    manu_embeddings = pickle.load(f)

In [27]:
len(manu_embeddings)

12075

In [22]:
with open("MSA_id_to_all_ids.pkl", "rb") as f:
    msa_to_id = pickle.load(f)

In [23]:
with open("propedia_data/embeddings.pkl", "rb") as f:
    propedia_embeddings = pickle.load(f)

In [24]:
len(propedia_embeddings)

19806

In [ ]:
for aa in manu_embed

In [25]:
combined_embeddings = manu_embeddings
for pdb_id in propedia_embeddings:
    partner_chain = f"{pdb_id.split('_')[0]}_{pdb_id.split('_')[2]}"
    if partner_chain not in combined_embeddings:
        combined_embeddings[partner_chain] = propedia_embeddings[pdb_id]


In [26]:
with open("embeddings.pkl", "wb") as f:
    pickle.dump(combined_embeddings, f)

In [41]:
combined_embeddings.keys()

dict_keys(['2r8e_A', '4rxh_A', '3qq0_A', '4ath_A', '1ssq_A', '3oj6_A', '4qyt_C', '4pca_A', '2qee_E', '3s1j_A', '1mr7_A', '2zbo_A', '1rli_B', '4hx6_A', '4upe_A', '4m1e_D', '2fpx_A', '1lt5_D', '4mzq_B', '1mrz_A', '4x1c_H', '3std_A', '3ijg_B', '4ulv_A', '2rld_A', '1dw9_E', '4zd6_B', '3mmz_B', '3c8w_A', '4dl1_B', '3lk4_S', '3vmz_A', '2jdy_B', '3mbh_A', '4fcc_I', '3uyu_A', '4crw_A', '3zyh_A', '2xdj_A', '2c2x_A', '3zop_D', '4ml1_B', '3v4m_A', '3wb0_A', '4p85_A', '3bbc_A', '4jo7_E', '4qtt_C', '4d2h_E', '2c94_A', '1gif_A', '4tty_A', '1x0r_C', '3dtt_A', '2zdh_A', '1kqd_A', '3i98_B', '4qbl_C', '3p9a_H', '1f60_A', '2wpq_B', '3pkz_A', '4d73_A', '2ra2_B', '3v2n_B', '1djr_F', '2iu7_E', '4cnn_A', '4d3h_B', '2wqj_A', '1tr0_F', '2r8e_F', '1q7l_A', '3k2x_A', '3oii_A', '3bf3_C', '3bvk_B', '2qtq_A', '4rt4_B', '1umn_E', '1zzg_A', '5azu_A', '4w8p_A', '2jgs_C', '1szo_B', '3nln_A', '3de8_A', '2qjn_A', '1nfv_M', '1dm5_D', '3kr4_A', '1wzg_A', '4yv4_C', '2q30_G', '2bos_C', '3wu2_b', '2otm_A', '3egv_A', '2wuj_A',

In [94]:
interface_errors = []
for i, row in combined_dataset.iterrows():
x        if isinstance(row['binding_region_indices'], str):
            indices = ast.literal_eval(row['binding_region_indices'])
        else:
            indices = row['binding_region_indices']
            
        indices = [int(ix) for ix in indices]
        partner_embedding = combined_embeddings[row['partner_chain']]

        if min(indices) < 0 or max(indices) >= partner_embedding.shape[1]:
            interface_errors.append(True)
        else:
            interface_errors.append(False)
    else:
        interface_errors.append(True)


In [95]:
sum(interface_errors)

3843

In [96]:
valid_interface = [not ix for ix in interface_errors]

In [97]:
combined_dataset_interface = combined_dataset[valid_interface]
combined_dataset_interface

,pdb_name,partner_chain,peptide_chain,peptide_seq,partner_seq,pdb_id,binding_region_indices,split
0,148l,148l_E,148l_S,AXXXX,MNIFEMLRIDEGLRLKIYKDTEGYYEIGIGHLLTKSPSLNAAKSEL...,148l_S_E,"[10, 17, 19, 20, 21, 23, 25, 28, 29, 30, 31, 3...",train
1,1a07,1a07_A,1a07_C,XEX,SIQAEEWYFGKITRRESERLLLNAENPRGTFLVRESETTKGAYCLS...,1a07_C_A,"[13, 33, 35, 36, 38, 42, 43, 44, 57, 58, 59, 6...",train
2,1a07,1a07_B,1a07_D,XEX,IQAEEWYFGKITRRESERLLLNAENPRGTFLVRESETTKGAYCLSV...,1a07_D_B,"[12, 32, 34, 35, 36, 37, 41, 42, 43, 56, 57, 5...",train
3,1a08,1a08_A,1a08_C,XEX,SIQAEEWYFGKITRRESERLLLNAENPRGTFLVRESETTKGAYCLS...,1a08_C_A,"[13, 16, 33, 34, 35, 36, 37, 38, 41, 42, 43, 4...",train
4,1a08,1a08_B,1a08_D,XEX,IQAEEWYFGKITRRESERLLLNAENPRGTFLVRESETTKGAYCLSV...,1a08_D_B,"[10, 11, 12, 32, 33, 34, 35, 36, 37, 40, 41, 4...",train
...,...,...,...,...,...,...,...,...
34969,4xcw,4xcw_D,4xcw_E,EATKKVCQKMLPGFGELMRM,QGMQTIHIGVLSASDRASYEDLSGKAIQEVLSEYLLNPLEFHYEIV...,4xcw_E_D,"[5, 31, 32, 35, 36, 58, 62, 63, 66, 67, 68, 69...",train
34970,2dvm,2dvm_C,2dvm_D,LEFHKNNFPGNGKIEVIPKV,IREKALEFHKNNFPGNGKIEVIPKVSLESREELTLAYTPGVAEPCK...,2dvm_D_C,"[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...",val
34971,2jbh,2jbh_A,2jbh_B,YGRGVVIMDDWPGYDLNLFT,EAPDYGRGVVIMDDWPGYDLNLFTYPQHYYGDLEYVLIPHGIIVDR...,2jbh_B_A,"[9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20...",train
34972,4kwt,4kwt_B,4kwt_C,KASTRTRCAFEVAAFDQGAQ,AMAFNLRNRNFLKLLDFSTKEIQFLIDLSADLKKAKYAGTEQKKLL...,4kwt_C_B,"[10, 24, 27, 28, 29, 30, 31, 32, 33, 34, 35, 3...",train


In [99]:
combined_dataset_interface.to_csv("pepgen_dataset_interface.csv")